In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
from sklearn.datasets import load_iris

In [3]:
iris = load_iris()

In [4]:
x = iris['data']
t = iris['target']

In [5]:
x = torch.tensor(x, dtype=torch.float32)
t = torch.tensor(t, dtype=torch.int64)

DataLoader

In [6]:
# 入力値と目標値をまとめる。lightgbmのDatasetと同じ
dataset = torch.utils.data.TensorDataset(x,t)

In [7]:
# train : val : test  = 6 : 2 : 2
n_train = int(len(dataset) * 0.6)
n_val = int(len(dataset) * 0.2)
n_test = len(dataset) - n_train - n_val

In [9]:
n_train, n_val, n_test

(90, 30, 30)

In [10]:
torch.manual_seed(0)

In [11]:
train, val, test = torch.utils.data.random_split(dataset, [n_train, n_val, n_test])

In [12]:
len(train),len(val),len(test),

(90, 30, 30)

## ミニバッチ学習

In [13]:
# バッチサイズの定義
batch_size = 10

In [14]:
train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True, drop_last=True)
val_loader = torch.utils.data.DataLoader(val, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test, batch_size=batch_size)

In [16]:
x, t = next(iter(train_loader))

In [17]:
x,t

(tensor([[7.7000, 3.8000, 6.7000, 2.2000],
         [5.0000, 3.4000, 1.6000, 0.4000],
         [5.5000, 3.5000, 1.3000, 0.2000],
         [6.4000, 2.8000, 5.6000, 2.2000],
         [6.7000, 2.5000, 5.8000, 1.8000],
         [7.7000, 3.0000, 6.1000, 2.3000],
         [5.1000, 2.5000, 3.0000, 1.1000],
         [5.0000, 3.6000, 1.4000, 0.2000],
         [7.3000, 2.9000, 6.3000, 1.8000],
         [6.1000, 2.8000, 4.0000, 1.3000]]),
 tensor([2, 0, 0, 2, 2, 2, 1, 0, 2, 1]))

## ネットワークの定義

### 4→4→3の全結合層を定義

In [18]:
class Net(nn.Module):
  # 使用するオブジェクトを定義
  def __init__(self):
    super().__init__()
    self.fc1 = nn.Linear(4,4)
    self.fc2 = nn.Linear(4,3)

  # 順伝播
  def forward(self, x):
    h = self.fc1(x)
    h = F.relu(h)
    h = self.fc2(h)
    return h

In [19]:
torch.manual_seed(0)
net = Net()

In [20]:
net

Net(
  (fc1): Linear(in_features=4, out_features=4, bias=True)
  (fc2): Linear(in_features=4, out_features=3, bias=True)
)

In [22]:
optimizer = torch.optim.SGD(net.parameters(), lr=0.01)

In [23]:
batch = next(iter(train_loader))
batch

[tensor([[5.4000, 3.9000, 1.7000, 0.4000],
         [4.6000, 3.6000, 1.0000, 0.2000],
         [6.5000, 3.0000, 5.5000, 1.8000],
         [6.9000, 3.1000, 5.4000, 2.1000],
         [6.3000, 2.5000, 4.9000, 1.5000],
         [7.1000, 3.0000, 5.9000, 2.1000],
         [5.8000, 2.7000, 4.1000, 1.0000],
         [7.0000, 3.2000, 4.7000, 1.4000],
         [6.7000, 3.0000, 5.0000, 1.7000],
         [7.2000, 3.6000, 6.1000, 2.5000]]),
 tensor([0, 0, 2, 2, 1, 2, 1, 1, 1, 2])]

In [24]:
x, t = batch

In [25]:
x

tensor([[5.4000, 3.9000, 1.7000, 0.4000],
        [4.6000, 3.6000, 1.0000, 0.2000],
        [6.5000, 3.0000, 5.5000, 1.8000],
        [6.9000, 3.1000, 5.4000, 2.1000],
        [6.3000, 2.5000, 4.9000, 1.5000],
        [7.1000, 3.0000, 5.9000, 2.1000],
        [5.8000, 2.7000, 4.1000, 1.0000],
        [7.0000, 3.2000, 4.7000, 1.4000],
        [6.7000, 3.0000, 5.0000, 1.7000],
        [7.2000, 3.6000, 6.1000, 2.5000]])

In [26]:
t

tensor([0, 0, 2, 2, 1, 2, 1, 1, 1, 2])

In [27]:
# 予測値の算出
y = net.forward(x)
y

tensor([[-0.2557, -0.2605, -0.4679],
        [-0.2041, -0.2834, -0.5574],
        [-0.2786, -0.2244, -0.3632],
        [-0.2552, -0.2214, -0.3703],
        [-0.3241, -0.2302, -0.3493],
        [-0.2788, -0.2244, -0.3631],
        [-0.3241, -0.2302, -0.3493],
        [-0.3241, -0.2302, -0.3493],
        [-0.3090, -0.2282, -0.3539],
        [-0.1884, -0.2129, -0.3907]], grad_fn=<AddmmBackward0>)

In [28]:
loss = F.cross_entropy(y,t)
loss

tensor(1.0882, grad_fn=<NllLossBackward0>)

In [29]:
loss.backward()
loss

tensor(1.0882, grad_fn=<NllLossBackward0>)

In [30]:
optimizer.step()

In [31]:
torch.cuda.is_available()

False

In [32]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [33]:
net.to(device)

Net(
  (fc1): Linear(in_features=4, out_features=4, bias=True)
  (fc2): Linear(in_features=4, out_features=3, bias=True)
)

In [34]:
x = x.to(device)
t = t.to(device)

## 実際の学習


In [46]:
# 勾配情報の初期化
optimizer.zero_grad()

In [36]:
# エポック数を１に設定
max_epoch = 1

In [43]:
torch.manual_seed(0)

In [44]:
net = Net().to(device)

In [45]:
# 最適化手法
optimizer = torch.optim.SGD(net.parameters(), lr=0.1)

In [47]:
# 学習のエポック数
for epoch in range(max_epoch):
  for batch in train_loader:

    x, t = batch
    x = x.to(device)
    t = t.to(device)

    y = net(x)

    loss = F.cross_entropy(y,t)

    y_label = torch.argmax(y, dim=1)
    accuracy = (y_label == t).sum().float() / len(t)
    print(f'accuracy : {accuracy:.2}')

    optimizer.zero_grad()
    loss.backward()

    optimizer.step()

accuracy : 0.6
accuracy : 0.8
accuracy : 0.6
accuracy : 0.6
accuracy : 0.6
accuracy : 0.5
accuracy : 0.4
accuracy : 0.2
accuracy : 0.4


# PyTorch Lightning

In [48]:
! pip install pytorch_lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 774.6/774.6 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 24.8 MB/s eta 0:00:00


In [49]:
import pytorch_lightning as pl

In [50]:
import torch
import torch.nn as nn
import torch.functional as F